## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [1]:
!pip3 install --upgrade pip

    100% |████████████████████████████████| 1.6MB 5.6MB/s eta 0:00:01    89% |████████████████████████████▋   | 1.4MB 14.0MB/s eta 0:00:01
  Found existing installation: pip 18.1
    Uninstalling pip-18.1:
      Successfully uninstalled pip-18.1


In [2]:
! pip3 install wheel

In [3]:
! pip install xgboost

     |████████████████████████████████| 166.7 MB 8.8 kB/s eta 0:00:011    |█████████████████████████       | 129.9 MB 9.8 MB/s eta 0:00:04     |█████████████████████████       | 130.5 MB 9.8 MB/s eta 0:00:04     |█████████████████████████▏      | 131.0 MB 9.8 MB/s eta 0:00:04     |█████████████████████████▎      | 131.6 MB 9.8 MB/s eta 0:00:04     |█████████████████████████▍      | 132.4 MB 9.8 MB/s eta 0:00:04     |█████████████████████████▌      | 133.0 MB 9.8 MB/s eta 0:00:04     |█████████████████████████▋      | 133.6 MB 9.8 MB/s eta 0:00:04     |█████████████████████████▊      | 134.3 MB 9.8 MB/s eta 0:00:04     |██████████████████████████      | 135.2 MB 9.8 MB/s eta 0:00:04     |██████████████████████████      | 135.9 MB 9.8 MB/s eta 0:00:04     |██████████████████████████▎     | 136.8 MB 9.8 MB/s eta 0:00:04     |██████████████████████████▍     | 137.6 MB 9.8 MB/s eta 0:00:03     |██████████████████████████▌     | 138.2 MB 9.8 MB/s eta 0:00:03     |██████████████████████████▋ 

In [4]:
!pip install --upgrade scikit-learn

     |████████████████████████████████| 22.2 MB 177 kB/s  eta 0:00:01   |▊                               | 481 kB 4.4 MB/s eta 0:00:05     |████                            | 2.9 MB 4.4 MB/s eta 0:00:05     |█████▏                          | 3.6 MB 4.4 MB/s eta 0:00:05     |███████▋                        | 5.3 MB 4.4 MB/s eta 0:00:04     |████████▋                       | 5.9 MB 4.4 MB/s eta 0:00:04     |█████████████████▋              | 12.2 MB 4.4 MB/s eta 0:00:03     |███████████████████▌            | 13.6 MB 11.3 MB/s eta 0:00:01     |████████████████████▌           | 14.3 MB 11.3 MB/s eta 0:00:01     |███████████████████████▏        | 16.1 MB 11.3 MB/s eta 0:00:01     |████████████████████████        | 16.7 MB 11.3 MB/s eta 0:00:01     |██████████████████████████▎     | 18.3 MB 11.3 MB/s eta 0:00:01     |████████████████████████████    | 19.5 MB 11.3 MB/s eta 0:00:01     |█████████████████████████████   | 20.1 MB 11.3 MB/s eta 0:00:01     |█████████████████████████████▉  | 20.7 MB

In [6]:
! pip install imblearn

     |████████████████████████████████| 189 kB 4.9 MB/s eta 0:00:01


In [7]:
# load in packages
#from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import xgboost as xgb

#import sklearn as sk

from imblearn.over_sampling import SMOTE
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report


import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head()


,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


In [8]:
train_data.shape


(84534, 10)

In [9]:
train_data.V1.value_counts()

1    31631
2    31608
3    10670
0    10625
Name: V1, dtype: int64

In [10]:
train_data.V4.value_counts()

2    57450
1    27084
Name: V4, dtype: int64

In [11]:
train_data.V5.value_counts()

3    32743
2    31196
1    15412
4     5183
Name: V5, dtype: int64

In [12]:
train_data.V6.value_counts()

3    21186
4    21176
2    21146
1    21026
Name: V6, dtype: int64

In [13]:
train_data.V7.value_counts()

2    59317
1    25217
Name: V7, dtype: int64

In [14]:
train_data.isnull().sum()

ID           0
Promotion    0
purchase     0
V1           0
V2           0
V3           0
V4           0
V5           0
V6           0
V7           0
dtype: int64

In [15]:
train_data.duplicated().sum()

0

In [16]:
train_data.ID.duplicated().sum()

0

In [17]:
train_data.Promotion.value_counts()

Yes    42364
No     42170
Name: Promotion, dtype: int64

In [18]:
train_data.purchase.value_counts()

0    83494
1     1040
Name: purchase, dtype: int64

In [19]:
train_data.query(" Promotion == 'Yes' and purchase == 1 ").sort_values(['ID'], ascending = False).reset_index().count()


index        721
ID           721
Promotion    721
purchase     721
V1           721
V2           721
V3           721
V4           721
V5           721
V6           721
V7           721
dtype: int64

In [20]:
train_data.query(" Promotion == 'No' and purchase == 0 ").sort_values(['ID'], ascending = False).reset_index().count()


index        41851
ID           41851
Promotion    41851
purchase     41851
V1           41851
V2           41851
V3           41851
V4           41851
V5           41851
V6           41851
V7           41851
dtype: int64

In [21]:
train_data.query(" Promotion == 'Yes' and purchase == 0 ").sort_values(['ID'], ascending = False).reset_index().count()


index        41643
ID           41643
Promotion    41643
purchase     41643
V1           41643
V2           41643
V3           41643
V4           41643
V5           41643
V6           41643
V7           41643
dtype: int64

In [22]:
train_data.query(" Promotion == 'No' and purchase == 1 ").sort_values(['ID'], ascending = False).reset_index().count()


index        319
ID           319
Promotion    319
purchase     319
V1           319
V2           319
V3           319
V4           319
V5           319
V6           319
V7           319
dtype: int64

In [23]:
response = []
for ele, row in train_data.iterrows():
    if row['Promotion'] == 'Yes' and row['purchase'] == 1:
        response.append(1)
    else:
        response.append(0)
        

In [24]:
train_data['response'] = response

In [25]:
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7,response
0,1,No,0,2,30.443518,-1.165083,1,1,3,2,0
1,3,No,0,3,32.159350,-0.645617,2,3,2,2,0
2,4,No,0,2,30.431659,0.133583,1,1,4,2,0
3,5,No,0,0,26.588914,-0.212728,2,1,4,2,0
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2,0


In [26]:
features = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7']
features


['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7']

In [27]:
x = train_data[features]
y = train_data['response']

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [29]:
x_train.head()

,V1,V2,V3,V4,V5,V6,V7
66588,3,34.470493,-1.078506,1,1,1,2
62125,1,28.065819,-1.165083,1,1,3,1
27827,1,27.142590,-1.511395,1,3,3,2
66648,3,28.782344,-1.078506,1,1,1,2
47392,2,39.605508,-0.645617,2,3,1,2


In [30]:
x_train.shape, y_train.shape

((67627, 7), (67627,))

In [31]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
X, Y = sm.fit_resample(x_train, y_train)

In [32]:
X.shape, Y.shape

((134094, 7), (134094,))

In [33]:
x_new_train = pd.DataFrame(X, columns = features)
y_new_train = pd.Series(Y)

In [34]:
x_new_train.head()

,V1,V2,V3,V4,V5,V6,V7
0,3,34.470493,-1.078506,1,1,1,2
1,1,28.065819,-1.165083,1,1,3,1
2,1,27.142590,-1.511395,1,3,3,2
3,3,28.782344,-1.078506,1,1,1,2
4,2,39.605508,-0.645617,2,3,1,2


from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_new_train = sc.fit_transform(x_new_train)
x_test = sc.transform(x_test)

## RandomForestClassifier

In [35]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state = 42, n_estimators = 500, max_depth = 5)
rfc.fit(x_new_train, y_new_train)

RandomForestClassifier(max_depth=5, n_estimators=500, random_state=42)

In [36]:
def promotion_strategy_2(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    y_pred = rfc.predict(df)
    
    promotion = []
    for ele in y_pred:
        if ele == 1:
            promotion.append('Yes')
        else:
            promotion.append('No')
            
    
    promotion = np.array(promotion)
    
    return promotion

In [37]:
promotion_strategy_2(x_test)

array(['No', 'Yes', 'No', ..., 'Yes', 'Yes', 'Yes'], dtype='<U3')

In [38]:
y_pred = rfc.predict(x_test)

print(classification_report(y_pred, y_test))


              precision    recall  f1-score   support

           0       0.61      0.99      0.76     10295
           1       0.49      0.01      0.02      6612

    accuracy                           0.61     16907
   macro avg       0.55      0.50      0.39     16907
weighted avg       0.56      0.61      0.47     16907



In [39]:
confusion_matrix(y_pred, y_test)

array([[10223,    72],
       [ 6543,    69]])

In [40]:
accuracy_score(y_pred, y_test)

0.6087419412077838

In [41]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

test_results(promotion_strategy_2)

Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is 0.0170.

Your nir with this strategy is 147.90.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


(0.01699907543508116, 147.9000000000001)

## GradientBoostingClassifier

In [42]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=600, learning_rate=1.0, max_depth=1, random_state=42).fit(x_new_train, y_new_train)

In [43]:
def promotion_strategy_3(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    y_pred = clf.predict(df)
    
    promotion = []
    for ele in y_pred:
        if ele == 1:
            promotion.append('Yes')
        else:
            promotion.append('No')
            
    
    promotion = np.array(promotion)
    
    return promotion

In [44]:
test_results(promotion_strategy_3)

Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is 0.0224.

Your nir with this strategy is 206.10.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


(0.02242820530186803, 206.10000000000002)

In [45]:
y_preds = clf.predict(x_test)

print(classification_report(y_preds, y_test))

              precision    recall  f1-score   support

           0       0.86      0.99      0.92     14492
           1       0.26      0.01      0.03      2415

    accuracy                           0.85     16907
   macro avg       0.56      0.50      0.47     16907
weighted avg       0.77      0.85      0.79     16907



In [46]:
confusion_matrix(y_preds, y_test)

array([[14387,   105],
       [ 2379,    36]])

In [47]:
accuracy_score(y_preds, y_test)

0.8530786064943514

## xgboostClassifier

In [48]:
import xgboost as xgb
eval_set = [(x_new_train, y_new_train), (x_test, y_test)]
model = xgb.XGBClassifier(learning_rate = 0.175,
                          max_depth = 7,
                          min_child_weight = 5,
                          objective = 'binary:logistic',
                          seed = 42,
                          gamma = 0.1,
                          silent = True)
model.fit(x_new_train, y_new_train, eval_set=eval_set, eval_metric="auc", verbose=True, early_stopping_rounds=30)

[01:21:32] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/opt/conda/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-auc:0.74608	validation_1-auc:0.56771
[1]	validation_0-auc:0.77237	validation_1-auc:0.57394
[2]	validation_0-auc:0.79474	validation_1-auc:0.56532
[3]	validation_0-auc:0.80033	validation_1-auc:0.57010
[4]	validation_0-auc:0.82849	validation_1-auc:0.57431
[5]	validation_0-auc:0.84010	validation_1-auc:0.56883
[6]	validation_0-auc:0.84707	validation_1-auc:0.57798
[7]	validation_0-auc:0.85470	validation_1-auc:0.57427
[8]	validation_0-auc:0.85755	validation_1-auc:0.57343
[9]	validation_0-auc:0.87009	validation_1-auc:0.57606
[10]	validation_0-auc:0.87699	validation_1-auc:0.57758
[11]	validation_0-auc:0.88188	validation_1-auc:0.57886
[12]	validation_0-auc:0.88630	validation_1-auc:0.58038
[13]	validation_0-auc:0.88920	validation_1-auc:0.58059
[14]	validation_0-auc:0.89921	validation_1-auc:0.57810
[15]	validation_0-auc:0.90384	validation_1-auc:0.57939
[16]	validation_0-auc:0.90473	validation_1-auc:0.57780
[17]	validation_0-auc:0.91485	validation_1-auc:0.57535
[18]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.1, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.175, max_delta_step=0, max_depth=7,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=1, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              silent=True, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [49]:
def promotion_strategy_1(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    y_pred = model.predict(df)
    
    promotion = []
    for ele in y_pred:
        if ele == 1:
            promotion.append('Yes')
        else:
            promotion.append('No')
            
    
    promotion = np.array(promotion)
    
    return promotion

In [50]:
test_results(promotion_strategy_1)


Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is 0.0211.

Your nir with this strategy is 341.85.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


(0.02109088396288788, 341.85)

In [51]:
yy_pred = model.predict(x_test)

print(classification_report(yy_pred, y_test))


              precision    recall  f1-score   support

           0       0.72      0.99      0.84     12212
           1       0.40      0.01      0.02      4695

    accuracy                           0.72     16907
   macro avg       0.56      0.50      0.43     16907
weighted avg       0.63      0.72      0.61     16907



In [52]:
confusion_matrix(yy_pred, y_test)

array([[12127,    85],
       [ 4639,    56]])

In [53]:
accuracy_score(yy_pred, y_test)

0.7205891051043947